# 원, 삼각형, 사각형 구분
- 어려운 테스트 문제를 기존 CNN 모델로 푸는 경우

In [2]:
!unzip '/content/drive/MyDrive/Colab Notebooks/DeepLearning_2022_05/hard_handwriting_shape.zip' -d handwriting > /dev/null

In [3]:
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt

In [4]:
shapes=['circle','rectangle','triangle']
for shape in shapes:
    dir_name = f'hard_handwriting/train/{shape}'
    plt.figure(figsize=(12,1))
    for i,fname in enumerate(glob(f'{dir_name}/*.png')):
        img=Image.open(fname)
        plt.subplot(1,15,i+1),plt.imshow(img),
        plt.xticks([]),plt.yticks([])
    plt.show()

<Figure size 864x72 with 0 Axes>

<Figure size 864x72 with 0 Axes>

<Figure size 864x72 with 0 Axes>

In [5]:
for shape in shapes:
    dir_name = f'hard_handwriting/test/{shape}'
    plt.figure(figsize=(4,1))
    for i,fname in enumerate(glob(f'{dir_name}/*.png')):
        img=Image.open(fname)
        plt.subplot(1,5,i+1),plt.imshow(img),
        plt.xticks([]),plt.yticks([])
    plt.show()

<Figure size 288x72 with 0 Axes>

<Figure size 288x72 with 0 Axes>

<Figure size 288x72 with 0 Axes>

In [ ]:
import numpy as np
import cv2
cv2.imread('handwriting/train/circle/circle001.png')[:,:,::-1].shape

(24, 24, 3)

- 데이터셋 생성

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1/255.)
train_data_generator=train_datagen.flow_from_directory(
    'handwriting/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'    
)

Found 45 images belonging to 3 classes.


In [7]:

test_datagen=ImageDataGenerator(rescale=1/255.)
test_data_generator=test_datagen.flow_from_directory(
    'handwriting/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'    
)

Found 15 images belonging to 3 classes.


In [8]:
train_data_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [9]:
train_data_generator.filenames[0]

'circle/circle001.png'

- 모델 정의/설정/학습/평가

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
seed=2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [11]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D,Flatten, MaxPooling2D, Dropout
#flatten - 3차원인 conv2D를 1차원인 dense, fully connected layer로 
#넣어야 하기 때문에 차원 축소를 할 때 필요.
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
model= Sequential([
                   Conv2D(32,(3,3),input_shape=(24,24,3),activation='relu'),
                    Conv2D(32,(3,3),activation='relu'),
                   MaxPooling2D(),
                    Flatten(),
                   Dense(128,activation='relu'),
                   Dense(3,activation='softmax')   
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 22, 22, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 10, 10, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3200)              0         
                                                                 
 dense (Dense)               (None, 128)               409728    
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                        

In [13]:
model.compile('adam','categorical_crossentropy',['accuracy'])

In [14]:
model.fit(
    train_data_generator,
    steps_per_epoch=15, #45/3
    epochs=50,
    validation_data=test_data_generator,
    validation_steps=5
    

)

Epoch 1/50
15/15 [==============================] - 12s 16ms/step - loss: 1.0161 - accuracy: 0.6444 - val_loss: 1.1232 - val_accuracy: 0.4000
Epoch 2/50
15/15 [==============================] - 0s 6ms/step - loss: 0.2672 - accuracy: 0.9556 - val_loss: 2.4955 - val_accuracy: 0.3333
Epoch 3/50
15/15 [==============================] - 0s 6ms/step - loss: 0.0273 - accuracy: 1.0000 - val_loss: 3.4787 - val_accuracy: 0.3333
Epoch 4/50
15/15 [==============================] - 0s 7ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 3.8521 - val_accuracy: 0.3333
Epoch 5/50
15/15 [==============================] - 0s 6ms/step - loss: 6.6653e-04 - accuracy: 1.0000 - val_loss: 4.0333 - val_accuracy: 0.3333
Epoch 6/50
15/15 [==============================] - 0s 6ms/step - loss: 4.8000e-04 - accuracy: 1.0000 - val_loss: 4.1297 - val_accuracy: 0.3333
Epoch 7/50
15/15 [==============================] - 0s 6ms/step - loss: 4.1098e-04 - accuracy: 1.0000 - val_loss: 4.1984 - val_accuracy: 0.3333
Epoch 

In [15]:
model.evaluate(test_data_generator,steps=5)

5/5 [==============================] - 0s 3ms/step - loss: 6.5321 - accuracy: 0.3333


[6.532107830047607, 0.3333333432674408]

In [16]:
model.evaluate(test_data_generator)

5/5 [==============================] - 0s 3ms/step - loss: 6.5321 - accuracy: 0.3333


[6.532107353210449, 0.3333333432674408]

In [19]:
model2= Sequential([
                   Conv2D(64,(3,3),input_shape=(24,24,3),activation='relu'),
                    Conv2D(64,(3,3),activation='relu'),
                   MaxPooling2D(),
                    Flatten(),
                   Dense(128,activation='relu'),
                   Dense(3,activation='softmax')   
])
model2.summary()
model2.compile('adam','categorical_crossentropy',['accuracy'])
model2.fit(
    train_data_generator,
    steps_per_epoch=15, #45/3
    epochs=50,
    validation_data=test_data_generator,
    validation_steps=5
    

)
model2.evaluate(test_data_generator,steps=5)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 22, 22, 64)        1792      
                                                                 
 conv2d_8 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 6400)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               819328    
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                      

[5.921248435974121, 0.3333333432674408]